In [1]:
import os
from PIL import Image

In [2]:
import numpy as np
import cv2

In [3]:
import pickle

In [4]:
BASE_DIR = os.path.dirname(os.path.abspath("__file__"))

In [5]:
image_dir =  os.path.join(BASE_DIR, "Images")

In [6]:
face_cascade = cv2.CascadeClassifier('Cascades/data/haarcascade_frontalface_alt2.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()

In [7]:
current_id = 0
y_labels = []

In [8]:
x_train = []

In [9]:
label_ids = {}

In [10]:
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith("png") or file.endswith("jpg"):
            path = os.path.join(root, file)
            label = os.path.basename(os.path.dirname(path)).replace(" ", "-").lower()
            #print(label, path)
            
            if not label in label_ids:
                label_ids[label] = current_id
                current_id += 1
            id_ = label_ids[label]
            #print(label_ids)
            
            pil_image = Image.open(path).convert("L") #Grayscale
            size =(550, 550)
            final_image = pil_image.resize(size, Image.ANTIALIAS)
            img_array = np.array(final_image, "uint8")
            #print(img_array)
            faces = face_cascade.detectMultiScale(img_array, scaleFactor = 1.5, minNeighbors=5)
            
            for (x,y,w,h) in faces:
                roi = img_array[y:y+h, x:x+w]
                x_train.append(roi)
                y_labels.append(id_)

In [11]:
print(x_train)

[array([[235, 238, 240, ..., 204, 195, 197],
       [237, 235, 237, ..., 204, 194, 194],
       [238, 235, 233, ..., 203, 196, 193],
       ...,
       [230, 234, 236, ..., 206, 204, 199],
       [231, 234, 235, ..., 206, 205, 201],
       [230, 233, 235, ..., 208, 201, 202]], dtype=uint8), array([[ 55,  58,  58, ...,  23,  27,  29],
       [ 70,  64,  83, ...,  24,  17,  17],
       [ 73,  81,  85, ...,  18,  23,  23],
       ...,
       [ 62,  68,  69, ..., 100,  84,  89],
       [ 63,  67,  67, ..., 108,  97,  78],
       [ 61,  57,  53, ..., 103, 102,  93]], dtype=uint8), array([[37, 40, 40, ..., 85, 87, 90],
       [37, 41, 41, ..., 84, 86, 89],
       [38, 40, 40, ..., 84, 86, 87],
       ...,
       [19, 19, 19, ..., 18, 20, 20],
       [21, 20, 19, ..., 19, 20, 19],
       [22, 21, 18, ..., 19, 19, 19]], dtype=uint8), array([[ 72,  77,  36, ...,  93,  88,  70],
       [116,  77,  60, ...,  48,  19,  39],
       [ 69, 104, 103, ..., 104,  70,  23],
       ...,
       [100, 101, 

In [12]:
print(y_labels)

[0, 0, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 7, 7, 7]


In [13]:
with open("labels.pickle", 'wb') as f:
    pickle.dump(label_ids, f)

In [14]:
recognizer.train(x_train, np.array(y_labels))
recognizer.save("Trainer.yml")